In [12]:
import pandas as pd
import glob
import os

FEATURE_COLUMNS = [
    'x_direction_changes', 'y_direction_changes', 'min_x', 'min_y', 'max_x', 'max_y',
    'bbox_area', 'avg_speed', 'avg_acceleration', 'mouse_idle_ratio',
    'movement_entropy', 'linearity'
]

def load_data_from_folder(folder_path, label):
    pattern = os.path.join(folder_path, "**", "*processed.csv")
    all_files = glob.glob(pattern, recursive=True)
    df_list = []
    for file in all_files:
        df = pd.read_csv(file, usecols=FEATURE_COLUMNS)
        df['label'] = label
        df_list.append(df)
    return pd.concat(df_list, ignore_index=True)


# Klasör yolları
REAL_DIR = r"C:\\store\\git\\km-stat-activity\\data\\real"
FAKE_DIR = r"C:\\store\\git\\km-stat-activity\\processed\\fake"

df_real = load_data_from_folder(REAL_DIR, label=0)
df_fake = load_data_from_folder(FAKE_DIR, label=1)
df_all = pd.concat([df_real, df_fake], ignore_index=True).sample(frac=1, random_state=42)

X = df_all[FEATURE_COLUMNS]
y = df_all['label']


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

# 1. Eğitim ve test verisine ayır
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# 2. Modeli oluştur
lr = LogisticRegression(max_iter=1000)

# 3. Modeli eğit
lr.fit(X_train, y_train)

# 4. Test verisi ile tahmin yap
y_pred_lr = lr.predict(X_test)

# 5. Confusion matrix ve metrikler
cm_lr = confusion_matrix(y_test, y_pred_lr)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr)
recall_lr = recall_score(y_test, y_pred_lr)
specificity_lr = cm_lr[0, 0] / (cm_lr[0, 0] + cm_lr[0, 1]) if (cm_lr[0, 0] + cm_lr[0, 1]) != 0 else 0

# 6. Sonuçları yazdır
print("Logistic Regression Confusion Matrix:\n", cm_lr)
print(f"Accuracy   : {accuracy_lr:.4f}")
print(f"Precision  : {precision_lr:.4f}")
print(f"Recall     : {recall_lr:.4f}")
print(f"Specificity: {specificity_lr:.4f}")


C:\Users\feyza\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Confusion Matrix:
 [[236958     45]
 [    13 180887]]
Accuracy   : 0.9999
Precision  : 0.9998
Recall     : 0.9999
Specificity: 0.9998
